In [2]:
import pandas as pd
import numpy as np
import ast
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem.porter import PorterStemmer

In [7]:
df = pd.read_excel('C:\dev\gen projects\Manga Recommendation App\Data\manga.xlsx')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67273 entries, 0 to 67272
Data columns (total 31 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   manga_id           67273 non-null  int64  
 1   title              67271 non-null  object 
 2   type               67273 non-null  object 
 3   score              21910 non-null  float64
 4   scored_by          67273 non-null  int64  
 5   status             67273 non-null  object 
 6   volumes            50494 non-null  float64
 7   chapters           47095 non-null  float64
 8   start_date         62949 non-null  float64
 9   end_date           51721 non-null  float64
 10  members            67273 non-null  int64  
 11  favorites          67273 non-null  int64  
 12  sfw                67273 non-null  bool   
 13  approved           67273 non-null  bool   
 14  created_at_before  67273 non-null  object 
 15  updated_at         58991 non-null  object 
 16  real_start_date    629

In [4]:
df.title_synonyms[df.title.isna()].apply(lambda x: ast.literal_eval(x)[0])

798      Plus Anima
38371      Plus One
Name: title_synonyms, dtype: object

In [8]:
# filling in the empty titles
df['ctitle'] = df.title
df.ctitle[df.title.isna()] = df.title_synonyms[df.title.isna()].apply(lambda x: ast.literal_eval(x)[-1])

<ipython-input-8-0faa445f5910>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.ctitle[df.title.isna()] = df.title_synonyms[df.title.isna()].apply(lambda x: ast.literal_eval(x)[-1])


In [9]:
df.ctitle.isna().sum()

0

In [10]:
df.ctitle.isnull().sum()

0

In [11]:
# checking for titles that are non roman texts
df[['ctitle','title_synonyms']][(df.title.apply(lambda x: str(x).isascii()) == False) & (df.title_synonyms != '[]')]

,ctitle,title_synonyms
68,Tokyo卍Revengers,['Tokyo Manji Revengers']
111,Kaze no Tani no Nausicaä,['Kaze no Tani no Nausicaa']
129,Lovely★Complex,"['Love Com Plus', 'Lovely Complex Plus']"
161,Shuumatsu no Walküre,"['Shuumatsu no Valkyrie', 'Valkyrie of the End']"
175,Yuu☆Yuu☆Hakusho,"['Yuu Yuu Hakusho', 'YuYu Hakusho', 'Two Shots']"
...,...,...
66970,Koryaku sareta mao beru wa akiramenai ！ Isekai...,['The defeated Demon King Bell will not give u...
66988,JSさんとOLちゃん,['Elementary School Girl and Office Lady']
67200,수상한메신저,['Mystic Messenger']
67259,Ai・Rabu・Yuu,['I Love You']


In [12]:
df['ctitle'][(df.ctitle.apply(lambda x: str(x).isascii()) == False) & (df.title_synonyms != '[]')] = df['title_synonyms'][(df.title.apply(lambda x: str(x).isascii()) == False) & (df.title_synonyms != '[]')].apply(lambda x: ast.literal_eval(x)[-1])

<ipython-input-12-ae1534ace837>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ctitle'][(df.ctitle.apply(lambda x: str(x).isascii()) == False) & (df.title_synonyms != '[]')] = df['title_synonyms'][(df.title.apply(lambda x: str(x).isascii()) == False) & (df.title_synonyms != '[]')].apply(lambda x: ast.literal_eval(x)[-1])


In [13]:
# checking for titles that are non roman texts
df[['title','ctitle','title_synonyms']][(df.ctitle.apply(lambda x: str(x).isascii()) == False) & (df.title_synonyms != '[]')]

,title,ctitle,title_synonyms
554,Hitoribocchi no ○○ Seikatsu,Hitori Bocchi's ○○ Lifestyle,"['Hitoribocchi no Marumaru Seikatsu', ""Hitori ..."
1707,MÄR,MÄR-Heaven,"['MAR', 'Märchen Awakens Romance', 'MAR-Heaven..."
1771,Yarichin☆Bitch-bu,Yarichin☆Bitch Club,['Yarichin☆Bitch Club']
1928,Abarenbou♥Darling,Abarenbou♥Kareshi,['Abarenbou♥Kareshi']
2059,Bokura no☆Hikari Club,Our☆Hikari Club,['Our☆Hikari Club']
...,...,...,...
66047,“Suki” ni ikiyou,“スキ” に 生きよう,['“スキ” に 生きよう']
66224,Arsène Lupin Kaitou Shinshi,Arsène Lupin Kaitō Shinshi,['Arsène Lupin Kaitō Shinshi']
66225,Arsène Lupin Kaitou Shinshi,Arsène Lupin Kaitō Shinshi,['Arsène Lupin Kaitō Shinshi']
66350,"Hello, Nice to Meet You. Let’s Divorce.",初めましてこんにちは、離婚してください,['初めましてこんにちは、離婚してください']


In [14]:
df.ctitle[df.ctitle.apply(lambda x: str(x).isascii()) == False] = df.title[df.ctitle.apply(lambda x: str(x).isascii()) == False]

<ipython-input-14-e1dedc1cb599>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.ctitle[df.ctitle.apply(lambda x: str(x).isascii()) == False] = df.title[df.ctitle.apply(lambda x: str(x).isascii()) == False]


In [15]:
# checking for titles that are non roman texts
df[['title','ctitle','title_synonyms']][(df.ctitle.apply(lambda x: str(x).isascii()) == False) & (df.title_synonyms != '[]')]

,title,ctitle,title_synonyms
554,Hitoribocchi no ○○ Seikatsu,Hitoribocchi no ○○ Seikatsu,"['Hitoribocchi no Marumaru Seikatsu', ""Hitori ..."
1707,MÄR,MÄR,"['MAR', 'Märchen Awakens Romance', 'MAR-Heaven..."
1771,Yarichin☆Bitch-bu,Yarichin☆Bitch-bu,['Yarichin☆Bitch Club']
1928,Abarenbou♥Darling,Abarenbou♥Darling,['Abarenbou♥Kareshi']
2059,Bokura no☆Hikari Club,Bokura no☆Hikari Club,['Our☆Hikari Club']
...,...,...,...
66047,“Suki” ni ikiyou,“Suki” ni ikiyou,['“スキ” に 生きよう']
66224,Arsène Lupin Kaitou Shinshi,Arsène Lupin Kaitou Shinshi,['Arsène Lupin Kaitō Shinshi']
66225,Arsène Lupin Kaitou Shinshi,Arsène Lupin Kaitou Shinshi,['Arsène Lupin Kaitō Shinshi']
66350,"Hello, Nice to Meet You. Let’s Divorce.","Hello, Nice to Meet You. Let’s Divorce.",['初めましてこんにちは、離婚してください']


### Creating the documents for the recommendation based on a combination of background and synopsis, genres and themes

In [36]:
df_manga_rel = df[['manga_id','ctitle','title_synonyms','synopsis','background','genres','themes']]
df_manga_rel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67273 entries, 0 to 67272
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   manga_id        67273 non-null  int64 
 1   ctitle          67273 non-null  object
 2   title_synonyms  67273 non-null  object
 3   synopsis        45619 non-null  object
 4   background      9339 non-null   object
 5   genres          67273 non-null  object
 6   themes          67273 non-null  object
dtypes: int64(1), object(6)
memory usage: 3.6+ MB


In [37]:
df_manga_rel.isna().sum()

manga_id              0
ctitle                0
title_synonyms        0
synopsis          21654
background        57934
genres                0
themes                0
dtype: int64

##### Combining synpsis and background

In [38]:
df_manga_rel[df_manga_rel.synopsis.isna() & df_manga_rel.background.isna()]

,manga_id,ctitle,title_synonyms,synopsis,background,genres,themes
1342,10376,Cavalier of the Abyss,[],NaN,NaN,"['Action', 'Adventure', 'Fantasy', 'Supernatur...",[]
1480,88110,Fate/stay night: Heaven's Feel,[],NaN,NaN,"['Action', 'Fantasy', 'Supernatural']",[]
1504,124483,Imaizumin Chi wa Douyara Gal no Tamariba ni Na...,[],NaN,NaN,['Hentai'],['Harem']
1536,57711,"Semete, Ano Yuki no You ni","['At Least, Like That Snow...']",NaN,NaN,"['Drama', 'Romance']",['School']
1712,103803,Melancholia,[],NaN,NaN,['Sci-Fi'],['Psychological']
...,...,...,...,...,...,...,...
67259,149649,I Love You,['I Love You'],NaN,NaN,"['Romance', 'Slice of Life', 'Sports']",['Romantic Subtext']
67261,149651,Uwaki to Honki,[],NaN,NaN,['Hentai'],[]
67266,149659,Little ☆ Enchanter,[],NaN,NaN,['Hentai'],[]
67267,149660,雨と一緒に,[],NaN,NaN,"['Romance', 'Supernatural']",[]


In [39]:
# fill the synopsis and background with empty space where empty
df_manga_rel.synopsis.fillna(' ', inplace=True)
df_manga_rel.background.fillna(' ', inplace=True)
df_manga_rel.isna().sum()

C:\Users\Ahmed\AppData\Roaming\Python\Python38\site-packages\pandas\core\series.py:4529: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


manga_id          0
ctitle            0
title_synonyms    0
synopsis          0
background        0
genres            0
themes            0
dtype: int64

In [40]:
(df_manga_rel.synopsis + ' ' + df_manga_rel.background).isna().sum()

0

In [41]:
# combining synopsis and background
df_manga_rel['description'] = df_manga_rel.synopsis + ' ' + df_manga_rel.background


<ipython-input-41-69d1a7b932b6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_manga_rel['description'] = df_manga_rel.synopsis + ' ' + df_manga_rel.background


In [42]:
df_manga_rel[df_manga_rel.description.isna()]

,manga_id,ctitle,title_synonyms,synopsis,background,genres,themes,description


In [43]:
df_manga_rel['description'].replace(' ', '', inplace=True)

C:\Users\Ahmed\AppData\Roaming\Python\Python38\site-packages\pandas\core\series.py:4575: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [44]:
df_manga_rel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67273 entries, 0 to 67272
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   manga_id        67273 non-null  int64 
 1   ctitle          67273 non-null  object
 2   title_synonyms  67273 non-null  object
 3   synopsis        67273 non-null  object
 4   background      67273 non-null  object
 5   genres          67273 non-null  object
 6   themes          67273 non-null  object
 7   description     67273 non-null  object
dtypes: int64(1), object(7)
memory usage: 4.1+ MB


In [45]:
df_manga_rel.isna().sum()

manga_id          0
ctitle            0
title_synonyms    0
synopsis          0
background        0
genres            0
themes            0
description       0
dtype: int64

##### process genres and themes

In [46]:
df_manga_rel[df_manga_rel.genres.isna() & df_manga_rel.themes.isna()]

,manga_id,ctitle,title_synonyms,synopsis,background,genres,themes,description


In [47]:
def process_genres_themes(x):
    return ' '.join('|'.join(ast.literal_eval(x)).replace(' ','').split('|'))

In [48]:
df_manga_rel.genres = df_manga_rel.genres.apply(process_genres_themes)
df_manga_rel.themes = df_manga_rel.themes.apply(process_genres_themes)
df_manga_rel.isna().sum()

C:\Users\Ahmed\AppData\Roaming\Python\Python38\site-packages\pandas\core\generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


manga_id          0
ctitle            0
title_synonyms    0
synopsis          0
background        0
genres            0
themes            0
description       0
dtype: int64

In [49]:
df_manga_rel['genres_and_themes'] = df_manga_rel['genres'] + ' ' + df_manga_rel['themes']
df_manga_rel['genres_and_themes']

<ipython-input-49-b87f89b9cd9f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_manga_rel['genres_and_themes'] = df_manga_rel['genres'] + ' ' + df_manga_rel['themes']


0        Action Adventure AwardWinning Drama Fantasy Ho...
1                                Action Adventure Fantasy 
2        Action Adventure AwardWinning Drama Fantasy Mi...
3        Action Adventure Horror Mystery Supernatural H...
4                          Drama SliceofLife Psychological
                               ...                        
67268                        Horror Supernatural Suspense 
67269                            Action Adventure Fantasy 
67270    BoysLove Fantasy Romance Historical Reincarnation
67271    BoysLove Drama Horror Romance Supernatural Psy...
67272                                            BoysLove 
Name: genres_and_themes, Length: 67273, dtype: object

In [50]:
df_manga_rel.isna().sum()

manga_id             0
ctitle               0
title_synonyms       0
synopsis             0
background           0
genres               0
themes               0
description          0
genres_and_themes    0
dtype: int64

### Creating a vectorizer

In [51]:
def process_description(text):
    # replace multiple spaces with one
    text = " ".join(text.split())
    stemer = PorterStemmer()

    # lowercase
    text = text.lower()
    text = stemer.stem(text)
    return text

def process_genresandthemes(text):
    # replace multiple spaces with one
    text = " ".join(text.split())

    # lowercase
    text = text.lower()
    return text

In [52]:
df_manga_rel['description_clean'] = df_manga_rel.description.apply(lambda x: process_description(x))

<ipython-input-52-e16985e15711>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_manga_rel['description_clean'] = df_manga_rel.description.apply(lambda x: process_description(x))


In [53]:
df_manga_rel['gt_clean'] = df_manga_rel.genres_and_themes.apply(lambda x: process_genresandthemes(x))

<ipython-input-53-5aecf9b353dc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_manga_rel['gt_clean'] = df_manga_rel.genres_and_themes.apply(lambda x: process_genresandthemes(x))


In [54]:
df_manga_rel.isna().sum()

manga_id             0
ctitle               0
title_synonyms       0
synopsis             0
background           0
genres               0
themes               0
description          0
genres_and_themes    0
description_clean    0
gt_clean             0
dtype: int64

In [55]:
df_manga_rel.isnull().sum()

manga_id             0
ctitle               0
title_synonyms       0
synopsis             0
background           0
genres               0
themes               0
description          0
genres_and_themes    0
description_clean    0
gt_clean             0
dtype: int64

In [62]:
df_manga_rel[df_manga_rel.genres_and_themes.isnull()]

,manga_id,ctitle,title_synonyms,synopsis,background,genres,themes,description,genres_and_themes,description_clean,gt_clean


In [56]:
df_manga_rel.to_csv('../artifacts/manga_clean.csv')

In [3]:
df_manga_rel_clean = pd.read_csv('../artifacts/manga_clean.csv')

In [4]:
df_manga_rel_clean.fillna('', inplace=True)

In [5]:
df_manga_rel_clean.isna().sum()

Unnamed: 0           0
manga_id             0
ctitle               0
title_synonyms       0
synopsis             0
background           0
genres               0
themes               0
description          0
genres_and_themes    0
description_clean    0
gt_clean             0
dtype: int64

In [25]:
import pickle
def save_object(filepath, object):
    objectrepo = open(filepath, 'wb')
    pickle.dump(object, objectrepo, protocol=pickle.HIGHEST_PROTOCOL)
    objectrepo.close()

In [22]:
# vectorizer for description will use tfidf
tf_idf = TfidfVectorizer(stop_words='english')
tf_idf_mat_desc = tf_idf.fit_transform(df_manga_rel['description_clean'])
tf_idf_mat_desc

<67273x85465 sparse matrix of type '<class 'numpy.float64'>'
	with 1568107 stored elements in Compressed Sparse Row format>

In [26]:
save_object('../artifacts/tfidf_desc.pkl', tf_idf_mat_desc)

In [11]:
from scipy import sparse

def opt_cos_sim(vect, batch_size, clip_value):
    num_batches = int(vect.shape[0]/batch_size)
    # results = np.zeros((vect.shape[0], vect.shape[0]))
    results = sparse.lil_matrix((vect.shape[0], vect.shape[0]))
    
    for i in range(num_batches):
        print(f'Batch {i}')
        start = i * batch_size
        end = (i+1) * batch_size
        arr = cosine_similarity(vect[start:end], vect)
        arr[arr < clip_value] = 0
        results[start:end] = arr
    
    rem = vect.shape[0]%batch_size
    results[-rem:] = cosine_similarity(vect[-rem:], vect)
    # results[results < 0.5] = 0
    # sparse_matrix = sparse.csr_matrix(results)

    return results


In [28]:
cos_simi_mat_desc = opt_cos_sim(tf_idf_mat_desc, 1000, 0.15)
cos_simi_mat_desc.size

5862474

In [36]:
cos_simi_mat_desc.shape

(67273, 67273)

In [39]:
cos_simi_mat_desc[0].todense()

matrix([[1., 0., 0., ..., 0., 0., 0.]])

In [27]:
#vectorizer for genres and themes will be a count vectorizer
count_vec = CountVectorizer(stop_words='english')
count_vec_mat_gt = count_vec.fit_transform(df_manga_rel_clean['gt_clean'])
count_vec_mat_gt

<67273x75 sparse matrix of type '<class 'numpy.int64'>'
	with 160777 stored elements in Compressed Sparse Row format>

In [13]:
cos_simi_mat_gt = opt_cos_sim(count_vec_mat_gt, 1000, 0.7)
cos_simi_mat_gt.shape

Batch 0
Batch 1
Batch 2
Batch 3
Batch 4
Batch 5
Batch 6
Batch 7
Batch 8
Batch 9
Batch 10


KeyboardInterrupt: 

In [ ]:
save_object('./artifacts/cosine_similarity_gt.pkl', cos_simi_mat_gt)

In [15]:
cos_simi_mat_gt = 0

In [6]:
import pickle
def read_object(filepath):
    obfile = open(filepath, 'rb')
    object = pickle.load(obfile)
    obfile.close()
    return object

In [7]:
def index_from_title(df,title):
    return df[df['ctitle'] == title].index.values[0]

def title_from_index(df, index):
    return df[df.index == index].title.values[0]

def recommendations(original_title, df, cosine_similarity_matrix, number_of_recommendations):
    index = index_from_title(df, original_title)
    similarity_scores = list(enumerate(np.squeeze(np.asarray(cosine_similarity_matrix[index].todense()))))
    # similarity_scores = list(enumerate(cosine_similarity_matrix[index]))
    similarity_scores_sorted = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    recommendations_indices = [t[0] for t in similarity_scores_sorted[1:(number_of_recommendations +1)]]
    return df['ctitle'].iloc[recommendations_indices]

In [8]:
cos_simi_mat_desc = read_object('../artifacts/cosine_similarity_desc.pkl')

/tmp/ipykernel_3840/3505564651.py:4: DeprecationWarning: Please use `lil_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.lil` namespace is deprecated.
  object = pickle.load(obfile)


In [9]:
df_manga_rel = pd.read_csv('../artifacts/manga_clean.csv', index_col='manga_id')

In [10]:
recommendations('Berserk', df_manga_rel, cos_simi_mat_desc, 10)

manga_id
4658          Fullmetal Alchemist
111271      Fullmetal Alchemist 0
21                     Death Note
401                      Kiseijuu
12                         Bleach
23390          Shingeki no Kyojin
149                        Blame!
3866                 Kuroshitsuji
698       Neon Genesis Evangelion
39883          Ansatsu Kyoushitsu
Name: ctitle, dtype: object